# Transformer based models

### Importing the required libraries

In [ ]:
import io
import numpy as np
# from google_trans_new import google_translator 
from scipy.spatial.distance import cosine
import json 
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import word_tokenize
from itertools import groupby 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U torch   
import torch
!pip install -U transformers
# !pip install -U sentence-transformers
import transformers
# from sentence_transformers import SentenceTransformer,util
import numpy as np

In [ ]:
import os
import json
import torch.nn.functional as F
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device:', device)

using device: cuda


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 9.1MB/s 


In [ ]:
          #### Sentence Transformer models ####
# model = SentenceTransformer('stsb-xlm-r-multilingual')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
# model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
# model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

In [ ]:
          #### Vanilla transformer models
import transformers
import sentencepiece
from transformers import AutoTokenizer, AutoModel

# config="xlm-roberta-large" 
# config="google/muril-base-cased"
config="ai4bharat/indic-bert"
# config="sentence-transformers/LaBSE"
# config="bert-base-multilingual-uncased"
# config="facebook/mbart-large-cc25" # also need to change

tokenizer = AutoTokenizer.from_pretrained(config)
model = AutoModel.from_pretrained(config).to(device)

ValueError: ignored

In [ ]:
### Using pooled output if required
import torch
import torch.nn as nn
model.eval()

def fact_str(fact, enable_qualifiers=False):
    fact_str = fact[1:3]
    qualifier_str = [' '.join(x) for x in fact[2]]
    if enable_qualifiers:
        fact_str.extend(qualifier_str)
    return fact_str

def pooled_rep(model_output, attention_mask, reduce='cls'):
    if reduce=='cls':
        return model_output[:, 0, :]
    elif reduce == "mean":
        token_embeddings = model_output #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
    elif reduce == 'sum':
        token_embeddings = model_output #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        return sum_embeddings
    else:
        raise Exception('reduce function not present !!!')

fact_list = ["I robbed bank","I like dogs"]
sentence_list = ["मुझे कुत्ते अच्छे लगते हैं"]
# fact_list = []
with torch.no_grad():
    enc = tokenizer.batch_encode_plus(sentence_list, padding='longest', return_attention_mask=True, return_tensors='pt')
    #taking the [CLS] token
    s_out = model(input_ids=enc["input_ids"].to(device), attention_mask=enc["attention_mask"].to(device))[0]
    sentence_encoding = pooled_rep(s_out, enc["attention_mask"].to(device), reduce='mean')
    
    # processed_facts = [' is '.join(fact_str(x)) for x in tfacts]
    fenc = tokenizer.batch_encode_plus(fact_list, padding='longest', return_attention_mask=True, return_tensors='pt')
    f_out = model(input_ids=fenc["input_ids"].to(device), attention_mask=fenc["attention_mask"].to(device))[0]
    facts_encoding = pooled_rep(f_out, fenc["attention_mask"].to(device), reduce='mean')


In [ ]:
scores = F.cosine_similarity(facts_encoding, sentence_encoding[0].unsqueeze(0), 1, 1e-6).cpu().tolist()
score_map = {i:v for i,v in enumerate(scores)}
score_map

{0: 0.4052985906600952, 1: 0.8921797275543213}

In [ ]:
fact_list = ["I robbed bank","I like dogs"]
sentence_list = ["मुझे कुत्ते अच्छे लगते हैं"]

In [ ]:
def get_sentence_specific_fact_alignment(tsentences, tfacts, index, threshold=0.5, score_type='cosine', reduce='cls'):
    res = []
    with torch.no_grad():
        enc = tokenizer.batch_encode_plus(tsentences, padding='longest', return_attention_mask=True, return_tensors='pt')
        #taking the [CLS] token
        s_out = model(input_ids=enc["input_ids"].to(device), attention_mask=enc["attention_mask"].to(device))[0]
        sentence_encoding = pooled_rep(s_out, enc["attention_mask"].to(device), reduce=reduce)
        
        
        processed_facts = [' is '.join(fact_str(x)) for x in tfacts]
        fenc = tokenizer.batch_encode_plus(processed_facts, padding='longest', return_attention_mask=True, return_tensors='pt')
        f_out = model(input_ids=fenc["input_ids"].to(device), attention_mask=fenc["attention_mask"].to(device))[0]
        facts_encoding = pooled_rep(f_out, fenc["attention_mask"].to(device), reduce=reduce)
        
        scores = F.cosine_similarity(facts_encoding, sentence_encoding[index].unsqueeze(0), 1, 1e-6).cpu().tolist()
        if score_type=='cosine':
            score_map = {i:v for i,v in enumerate(scores)}
        else:
            neighbour = 5
            sentence_neighbours_enc = F.cosine_similarity(sentence_encoding, sentence_encoding[index].unsqueeze(0))
            sentence_scores = [x.item() for i, x in enumerate(sentence_neighbours_enc) if i!=index]
            sent_k_score = max(np.sum(sentence_scores[:neighbour]), 0)
            sent_k = max(len(sentence_scores[:neighbour]), 1)
            
            score_map = {}
            for i, _ in enumerate(tfacts):
                temp_fact_cosine = F.cosine_similarity(facts_encoding, facts_encoding[i].unsqueeze(0))
                facts_scores = [x.item() for j, x in enumerate(temp_fact_cosine) if i!=j]
                fact_k_score = max(np.sum(facts_scores[:neighbour]), 0)
                fact_k = max(len(facts_scores[:neighbour]), 1)
                denom = (fact_k_score/(2*fact_k)) + (sent_k_score/(2*sent_k))
                score_map[i] = scores[i]/denom
        
        max_facts = int(threshold*len(score_map))
        # print(score_map)
        for j, u in sorted(score_map.items(), key=lambda x: x[1], reverse=True):
            if max_facts<1:
                break
            res.append(j)
            max_facts-=1
        return res
index = 0
pred_fact_index = get_sentence_specific_fact_alignment(sentence_list, fact_list, index, threshold=0.5, score_type='cosine', reduce='mean')
predicted_facts = [fact_list[index] for index in pred_fact_index]
print(sentence_list[index])
predicted_facts

मुझे कुत्ते अच्छे लगते हैं


['I like dogs']

In [ ]:
#@title
sentence1 = "I robbed bank"
sentence2 = "मुझे कुत्ते अच्छे लगते हैं"
# encode sentences to get their embeddings
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores.item())

AttributeError: ignored

### Loading data from json file

In [ ]:
import json 
# Opening JSON file 
testA,testC,testP = open('/content/drive/MyDrive/test_data/testA.json',encoding = 'utf-8') ,open('/content/drive/MyDrive/test_data/testC2.json',encoding = 'utf-8'),open('/content/drive/MyDrive/test_data/testP.json',encoding = 'utf-8')
dataA,dataC,dataP = json.load(testA),json.load(testC),json.load(testP),
# returns JSON object as  
# a dictionary 
# data = json.load(f) 
# d = {}
# d = data
  
# Closing file 
# f.close()

In [ ]:
for qid in dataC:
  t =  dataC[qid]
  for key in t:
    if key == 'triples':
      triplelist = t[key]
      for l in triplelist:
        if len(l) == 1:
          triplelist.remove(l)

In [ ]:
### Getting a_c_p for getting all the triples
# Opening JSON file 
f = open('/content/drive/MyDrive/a_c_p.json',) 
data = json.load(f) 
acp = {}
acp = data
f.close() 

### Storing sentences and triples from one article
Here we store all the sentences and the triples from one article. The article is about the actress 'Kalpana'. We display 5 sentences and 5 triples corresponding to her here.

### Obtaining sentence and triple embeddings
We obtain the sentence embeddings by taking each sentence, obtaining word embeddings for each word in the sentence, and averaging the word embeddings. For triple embeddings , we average the embeddings for each word in the triple.

In [ ]:
#@title
# Matching Triples with sentences
actors_test, cricketers_test, politicians_test = dataA, dataC, dataP
actors, cricketers, politicians = actors_test, cricketers_test, politicians_test

# Creating the dictionary for the test data where key = sentence and value = list of matching triples
actors_test_dict, cricketers_test_dict, politicians_test_dict = {}, {}, {}
for l in actors_test:
    for k, v in actors_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    actors_test_dict[sentence] = set(t)

for l in cricketers_test:
    for k, v in cricketers_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    cricketers_test_dict[sentence] = set(t)

for l in politicians_test:
    for k, v in politicians_test[l].items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    politicians_test_dict[sentence] = set(t)
    
len(actors_test_dict),len(cricketers_test_dict),len(politicians_test_dict)

In [ ]:
#@title
# We had got a_c_p.json. We keep only the relevant triples by filtering by entity id in test annotated data
actors_trip, cricketers_trip, politician_trip = [], [], []
actors_sent, cricketers_sent, politician_sent = [], [], []

# Putting actors,cricketers and politicians from a_c_p
act, cric, pol = acp['a'], acp['c'], acp['p']

############### Actors #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in actors_test:
    eid = actors_test[e]['entity_id']
    for ele in act:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = act[ele]['triples']
            subject = act[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            actors_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in actors_test:
        if eid == actors_test[e]['entity_id']:
            sentence = actors_test[e]['sentence']
            sentence_list.append(sentence)
    actors_sent.append(sentence_list)


############### Cricketers #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in cricketers_test:
    eid = cricketers_test[e]['entity_id']
    for ele in cric:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = cric[ele]['triples']
            subject = cric[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            cricketers_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in cricketers_test:
        if eid == cricketers_test[e]['entity_id']:
            sentence = cricketers_test[e]['sentence']
            sentence_list.append(sentence)
    cricketers_sent.append(sentence_list)


############### Politicians #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in politicians_test:
    eid = politicians_test[e]['entity_id']
    for ele in pol:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = pol[ele]['triples']
            subject = pol[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            politician_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in politicians_test:
        if eid == politicians_test[e]['entity_id']:
            sentence = politicians_test[e]['sentence']
            sentence_list.append(sentence)
    politician_sent.append(sentence_list)

In [ ]:
len(actors_sent), len(cricketers_sent),len(politician_sent)

In [ ]:
len(actors_trip), len(cricketers_trip),len(politician_trip)

In [ ]:
#@title
def matches(sentences,triples):       # Input is a sentence list and triple list for one article
    matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
    for sent in sentences:            # For each sent in sentences
        ent_matchlist = []
        sent_embed = model.encode(sent, convert_to_tensor=True)
        for ent in triples:
            pred = ent[1]
            obj = ent[2]
            # print(pred)
            ent_embed = model.encode(pred + " " + obj, convert_to_tensor=True)
            cosine_score = util.pytorch_cos_sim(sent_embed, ent_embed)
            similarity = cosine_score.item()
            
            if similarity > 0.2:
              # For evaluation of precision and recall, keep the below 3 lines commented out. They are to append score to matching triples
                # ent = list(ent)
                # ent.append(similarity)
                # ent = tuple(ent)
                ent_matchlist.append(ent)
        if len(ent_matchlist)>0:    
            matches_dict[sent] = set(ent_matchlist)
    return matches_dict

In [ ]:
#@title
# #@title
#                           ###### Sentence Trasnformer matches ############

# matches_act = {}
# for sent_list, triple_list in tqdm(zip(actors_sent, actors_trip)):
#     for k, v in matches(sent_list, triple_list).items():
#         matches_act[k] = v

# matches_cric = {}
# for sent_list, triple_list in tqdm(zip(cricketers_sent, cricketers_trip)):
#     for k, v in matches(sent_list, triple_list).items():
#         matches_cric[k] = v

# matches_pol = {}
# for sent_list, triple_list in tqdm(zip(politician_sent, politician_trip)):
#     for k, v in matches(sent_list, triple_list).items():
#         matches_pol[k] = v

In [ ]:
                      ############ Vanilla transformer matches ##########
actors_sent_univ = [slist[0] for slist in actors_sent]
cricketers_sent_univ = [slist[0] for slist in cricketers_sent]
politicians_sent_univ = [slist[0] for slist in politician_sent]

pred_actors_indexes = [ get_sentence_specific_fact_alignment(actors_sent_univ,actors_trip[index],index,
                                                                 threshold = 0.5, score_type = 'cosine',
                                                                 reduce = 'mean')
                        for index in range(len(actors_sent_univ))
                      ]
predicted_facts = [ list(map(actors_trip[i].__getitem__ ,pred_actors_indexes[i])) for i in range(len(actors_trip))]
matches_actors = {i:v for i,v in zip(actors_sent_univ,predicted_facts)}

pred_cricketers_indexes = [ get_sentence_specific_fact_alignment(cricketers_sent_univ,cricketers_trip[index],index,
                                                                 threshold = 0.5, score_type = 'cosine',
                                                                 reduce = 'mean')
                        for index in range(len(cricketers_sent_univ))
                      ]

predicted_facts = [ list(map(cricketers_trip[i].__getitem__ ,pred_cricketers_indexes[i])) for i in range(len(cricketers_trip))]
matches_cricketers = {i:v for i,v in zip(cricketers_sent_univ,predicted_facts)}

pred_politicians_indexes = [ get_sentence_specific_fact_alignment(politicians_sent_univ,politician_trip[index],index,
                                                                 threshold = 0.5, score_type = 'cosine',
                                                                 reduce = 'mean')
                        for index in range(len(politicians_sent_univ))
                      ]
predicted_facts = [ list(map(politician_trip[i].__getitem__ ,pred_politicians_indexes[i])) for i in range(len(politician_trip))]
matches_politicians = {i:v for i,v in zip(politicians_sent_univ,predicted_facts)}


### Evaluation : Precision and Recall

In [ ]:
def evaluate(test_dict, matches_dict):
    sum_prec = 0
    for key, val in matches_dict.items():
        tp, fp = 0, 0
        for k, v in test_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        if (tp+fp) != 0:
            prec = tp/(tp + fp)
        else:
            prec = 0
        sum_prec = prec + sum_prec

    sum_rec = 0
    for k, v in test_dict.items():
        rec = 0
        tp, fp = 0, 0
        for key, val in matches_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        rec = tp/len(v)
        sum_rec = rec + sum_rec

    avg_rec, avg_prec = sum_rec/len(test_dict), sum_prec/len(matches_dict)
    return avg_rec, avg_prec

avg_rec_act, avg_prec_act = evaluate(actors_test_dict, matches_actors)
avg_rec_cric, avg_prec_cric = evaluate(cricketers_test_dict, matches_cricketers)
avg_rec_pol, avg_prec_pol = evaluate(politicians_test_dict, matches_politicianss)
# avg_rec_act, avg_prec_act = evaluate(actors_test_dict, matches_act)
# avg_rec_cric, avg_prec_cric = evaluate(cricketers_test_dict, matches_cric)
# avg_rec_pol, avg_prec_pol = evaluate(politicians_test_dict, matches_pol)

In [ ]:
(avg_rec_act,avg_prec_act)

In [ ]:
(avg_rec_act,avg_prec_act), (avg_rec_cric, avg_prec_cric) , (avg_rec_pol, avg_prec_pol)

In [ ]:
AverageRecall = (avg_rec_act + avg_rec_cric + avg_rec_pol)/3
AveragePrecision = (avg_prec_act + avg_prec_cric + avg_prec_pol)/3

AverageRecall, AveragePrecision

(0.38040873015873006, 0.21436843603020073)

In [ ]:
matches_pol

### Output

In [ ]:
len(matches_act),len(matches_cric),len(matches_pol)

So, 7 sentences didn't find any matches at all.Let's see which ones

In [ ]:
for sent in politicians_test_dict:
  if sent not in matches_pol:
    print(sent," : ", politicians_test_dict[sent] )

Saving as json

In [ ]:
for sent in actors_test_dict:
  actors_test_dict[sent] = list(actors_test_dict[sent])
for sent in cricketers_test_dict:
  cricketers_test_dict[sent] = list(cricketers_test_dict[sent])
for sent in politicians_test_dict:
  politicians_test_dict[sent] = list(politicians_test_dict[sent])

for sent in matches_act:
  matches_act[sent] = list(matches_act[sent])
for sent in matches_cric:
  matches_cric[sent] = list(matches_cric[sent])
for sent in matches_pol:
  matches_pol[sent] = list(matches_pol[sent])

In [ ]:
# saving the input test files
with open("/content//drive//MyDrive//Transformer Output//actors_test.json", "w") as outfile: 
    json.dump(actors_test_dict,outfile)
with open("/content/drive/MyDrive/Transformer Output/cricketers_test.json", "w") as outfile: 
    json.dump(cricketers_test_dict, outfile)
with open("/content/drive/MyDrive/Transformer Output/politicians_test.json", "w") as outfile: 
    json.dump(politicians_test_dict, outfile)

#saving the output files
with open("/content/drive/MyDrive/Transformer Output/actors_matches.json", "w") as outfile: 
    json.dump(matches_act, outfile)
with open("/content/drive/MyDrive/Transformer Output/cricketers_matches.json", "w") as outfile: 
    json.dump(matches_cric, outfile)
with open("/content/drive/MyDrive/Transformer Output/politicians_matches.json", "w") as outfile: 
    json.dump(matches_pol, outfile)

#### We do find quite a few of the matching triples to be relevant to the sentence. But, there are a few irrelevant matches as well.
Upon analysis, we think the word overlap is working better than the vector similarity approach. A possible reason can be that when we simply average out the words in a sentence, and when we average out the words in the triples and then find the similarity between these two averages, some semantic information is lost. So, triples that should have been irrelevant are also found as similar. As the word overlap method is a strictly string overlap, the relevance is much stronger.